#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/competitions/prediccion-precios-laptops/overview

## Librerias

In [704]:
import pandas as pd
import numpy as np
import urllib.request
from PIL import Image
import re
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

## Data

In [705]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
train = pd.read_csv("data/train.csv")


## Data exploration

In [706]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         912 non-null    int64  
 1   Company           912 non-null    object 
 2   Product           912 non-null    object 
 3   TypeName          912 non-null    object 
 4   Inches            912 non-null    float64
 5   ScreenResolution  912 non-null    object 
 6   Cpu               912 non-null    object 
 7   Ram               912 non-null    object 
 8   Memory            912 non-null    object 
 9   Gpu               912 non-null    object 
 10  OpSys             912 non-null    object 
 11  Weight            912 non-null    object 
 12  Price_euros       912 non-null    float64
dtypes: float64(2), int64(1), object(10)
memory usage: 92.8+ KB


In [707]:
train.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1104,Acer,Aspire ES1-523,Notebook,15.6,1366x768,AMD A8-Series 7410 2.2GHz,4GB,500GB HDD,AMD Radeon R5,Windows 10,2.4kg,387.0
1,114,Dell,XPS 13,Ultrabook,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i7 7560U 2.4GHz,8GB,256GB SSD,Intel Iris Plus Graphics 640,Windows 10,1.23kg,1379.0
2,172,Acer,Aspire A517-51G,Notebook,17.3,IPS Panel Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,Nvidia GeForce MX150,Windows 10,3kg,854.0
3,918,HP,Elitebook 820,Netbook,12.5,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.26kg,1483.0
4,447,Lenovo,IdeaPad 320-15AST,Notebook,17.3,1600x900,AMD A6-Series 9220 2.5GHz,8GB,1TB HDD,AMD Radeon R4,Windows 10,2.8kg,519.0


In [708]:
# train['OpSys'].unique() # Agrupar y separar en columnas
# train['Gpu'].unique()
# train['Cpu'].unique() # Se puede limpiar y nos quedamos con fabricante core o serie y velocidad
# train['Memory'].unique() # Separar entre SHDD, SSD Y HDD y luego otra columna con la capacidad de cada uno

In [709]:
def unifica_ssoo(df: pd.DataFrame) -> pd.DataFrame:
    df['OpSys'] = df['OpSys'].str.replace("Windows 10 S", "Windows 10")
    df['OpSys'] = df['OpSys'].str.replace("Mac OS X", "macOS")

    return df

In [710]:
def apply_onehot_encoder(train:pd.DataFrame, columns_to_encode:list, test:pd.DataFrame=None):
    
    # Resetear índices para evitar desalineación
    train = train.reset_index(drop=True)
    
    # Crear el OneHotEncoder
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

    # Ajustar y transformar las columnas seleccionadas
    transformed_data = encoder.fit_transform(train[columns_to_encode])

    # Crear un DataFrame con las columnas transformadas
    transformed_df = pd.DataFrame(transformed_data, columns=encoder.get_feature_names_out(columns_to_encode))
    
    # Concatenar con el DataFrame original excluyendo las columnas transformadas
    df_concatenated = pd.concat([train.drop(columns_to_encode, axis=1), transformed_df], axis=1)

    # Si se proporciona un segundo DataFrame, aplicar la misma transformación
    if test is not None:
        transformed_data_to_transform = encoder.transform(test[columns_to_encode])
        transformed_df_to_transform = pd.DataFrame(transformed_data_to_transform, columns=encoder.get_feature_names_out(columns_to_encode))
        df_to_transform_concatenated = pd.concat([test.drop(columns_to_encode, axis=1), transformed_df_to_transform], axis=1)
        return df_concatenated, df_to_transform_concatenated

    return df_concatenated

## Data processing

Voy a pasar los valores que puedo a float simplemente eliminando la parte no numerica de la columna

In [711]:
def limpia_peso(df: pd.DataFrame) -> pd.DataFrame:
    df['Weight'] = df['Weight'].str.replace("kg", "")
    df['Weight'] = df['Weight'].astype(float)

    return df

In [712]:
def limpia_ram(df: pd.DataFrame) -> pd.DataFrame:
    df['Ram'] = df['Ram'].str.replace("GB", "")
    df['Ram'] = df['Ram'].astype(int)

    return df

In [713]:
# print(train['Memory'].unique())

def de_tb_a_gb(capacidad: str):
    if len(capacidad) > 1:
        if capacidad[1] == 'T':
            return int(float(capacidad[0]) * 1000)
        else:
            return int(capacidad[0])
    else:
        if capacidad[0][1] == 'T':
            return int(float(capacidad[0][0]) * 1000)
        else:
            return int(capacidad[0][0])

def tipo_rom_value(tipo_disco):
    if "SSD" == tipo_disco:
        return 4
    elif "Flash Storage" == tipo_disco:
        return 4
    elif "Hybrid" == tipo_disco:
        return 2
    elif "HDD" == tipo_disco:
        return 1
    else:
        return 0
    

def limpia_rom(df: pd.DataFrame) -> pd.DataFrame:
    main_storage_type = []
    main_storage_capacity = []
    secondary_storage_type = []
    secondary_storage_capacity = []

    for memoria in df['Memory']:
        capacidades_discos = re.findall(r"(\d+(?:\.\d+)?)([GB|TB])", memoria)
        tipos_discos = re.findall(r"(SSD|HDD|Hybrid|Flash Storage)", memoria)
        if len(capacidades_discos) > 1:
            main_storage_type.append(tipo_rom_value(tipos_discos[0]))
            main_storage_capacity.append(de_tb_a_gb(capacidades_discos[0]))
            secondary_storage_type.append(tipo_rom_value(tipos_discos[1]))
            secondary_storage_capacity.append(de_tb_a_gb(capacidades_discos[1]))
        else:
            main_storage_type.append(tipo_rom_value(tipos_discos[0]))
            main_storage_capacity.append(de_tb_a_gb(capacidades_discos))
            secondary_storage_type.append(0)
            secondary_storage_capacity.append(0)


    df['main_storage_type'] = main_storage_type
    df['main_storage_capacity'] = main_storage_capacity

    df['secondary_storage_type'] = secondary_storage_type
    df['secondary_storage_capacity'] = secondary_storage_capacity

    return df

In [714]:
def limpia_screen(df: pd.DataFrame) -> pd.DataFrame:
    resoluciones = []

    df["screen_ips"] = df["ScreenResolution"].str.find("IPS") != -1
    df["Touchscreen"] = df["ScreenResolution"].str.find("Touchscreen") != -1

    elementos_resolucion = [string.split(" ") for string in df["ScreenResolution"]]

    for index, elemento in enumerate(elementos_resolucion):
        for dato in elemento:
            if str(dato).find("x") != -1:
                valores = re.findall(r"\d+", dato)
                # Multiplicar las resoluciones
                resoluciones.append(int(valores[0]) * int(valores[1]))

    df['Resolution'] = resoluciones

    # le = LabelEncoder()
    # le.fit(df["Resolution"])
    # df["Resolution"] = le.transform(df["Resolution"])

    df = df.drop("ScreenResolution", axis=1)

    return df

In [715]:
def limpia_cpu(df: pd.DataFrame) -> pd.DataFrame:
    df['Cpu'] = df['Cpu'].str.replace(" Core", "")
    df['Cpu'] = df['Cpu'].str.replace(" Quad", "")
    df['Cpu'] = df['Cpu'].str.replace(" Dual", "")

    elementos_cpu = [string.split(" ") for string in df["Cpu"]]

    cpu_fabricante = []
    cpu_core = []
    cpu_modelo = []
    cpu_velocidad = []

    # Comprobar si los elementos contienen un caracter o palabra concreta
    for index, elemento in enumerate(elementos_cpu): 
        if len(elemento) != 4:
            if (elemento[1] == 'Xeon'):
                elemento[2] = elemento[2] + '_' + elemento[3]
                elemento.pop(3)
            else:
                elemento.insert(-1, "no_model")


        if elemento[1] == r"i7|Xeon|FX|Ryzen|A8-Series":
            cpu_core.append(4)
        elif elemento[1] == r"i5|A10-Series|A6-Series|A12-Series":
            cpu_core.append(3)
        elif elemento[1] == r"i3|A9-Series|Celeron|E-Series":
            cpu_core.append(2)
        else:
            cpu_core.append(1)

            
        cpu_fabricante.append(elemento[0])
        cpu_modelo.append(elemento[2])
        cpu_velocidad.append(elemento[3].replace("GHz", ""))

    df['cpu_brand'] = cpu_fabricante
    df['cpu_core'] = cpu_core
    df['cpu_model'] = cpu_modelo
    # le = LabelEncoder()
    # le.fit(df["cpu_model"])
    # df["cpu_model"] = le.transform(df["cpu_model"])
    df['cpu_speed'] = cpu_velocidad

    df = df.drop("Cpu", axis=1)

    return df

In [716]:
def busca_grafica_dedicada(df: pd.DataFrame) -> pd.DataFrame:
    df["dedicated_gpu"] = df['Gpu'].str.contains(r"(Nvidia|AMD)")

    return df

In [717]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

columns_to_exclude = ["laptop_ID","Price_euros", 'Company', 'TypeName', 'OpSys', 'cpu_brand', 'cpu_model']
# columns_to_exclude = ["laptop_ID","Price_euros", 'Company', 'TypeName', 'OpSys']

def func_exclude_columns(X, exclude_cols = columns_to_exclude):
        # Skip missing values from colum
        X_temp = X[[col for col in X.columns if col not in exclude_cols]].copy()
        return X_temp

def func_StandardScaler(X, cols_to_scale):
        X_temp = X.copy()
        SScaler = StandardScaler()
        SScaler.fit(X_temp[cols_to_scale])
        X_temp[cols_to_scale] = SScaler.transform(X_temp[cols_to_scale])
        return X_temp

def estandarizar(df:pd.DataFrame) -> pd.DataFrame:
    # Column Transformer nos permite aplicar diferentes transformers según las columnas
    pipe_exclude_columns = ColumnTransformer([("excluye","drop", columns_to_exclude)], remainder = "passthrough")

    pipe_exclude_columns.get_params()

    train_compacto = func_exclude_columns(df, columns_to_exclude)

    return func_StandardScaler(df, train_compacto.columns)

In [718]:
def limpiar_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    df = unifica_ssoo(df)
    df = limpia_peso(df)
    df = limpia_ram(df)
    df = limpia_rom(df)
    df = limpia_screen(df)
    df = limpia_cpu(df)
    df = limpia_rom(df)
    df = busca_grafica_dedicada(df)

    df = df.drop(["Product", "Memory", "Gpu"], axis=1)

    df = estandarizar(df)
    
    return df

In [719]:
train = limpiar_dataframe(train)

C:\Users\cgdo2\AppData\Local\Temp\ipykernel_75352\2141475305.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["dedicated_gpu"] = df['Gpu'].str.contains(r"(Nvidia|AMD)")


-----------------------------------------------------------------------------------------------------------------

In [720]:
train2 = apply_onehot_encoder(train, ["Company", "TypeName", "OpSys", "cpu_brand", "cpu_model"])

c:\Users\cgdo2\Documents\Master Data\DS_PT_09_2023\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Pasos que si o si debéis realizar para poder participar en la competición:

### 1. Definir X e y

In [721]:
X = train2.drop(['Price_euros'], axis=1)
y = train2['Price_euros'].copy()
X.shape

(912, 123)

In [722]:
y.shape

(912,)

### 2. Dividir X_train, X_test, y_train, y_test

In [723]:
from sklearn.model_selection import train_test_split

In [724]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [725]:
from sklearn.model_selection import cross_val_score

In [726]:
# from sklearn.linear_model import LogisticRegression



# x = train2.drop(['Price_euros'], axis=1)
# y = train2['Price_euros'].copy()

# from sklearn.preprocessing import LabelEncoder

# # Discretiza las etiquetas
# etiquetas_discretizadas = LabelEncoder().fit_transform(y)

# modelo = LogisticRegression()
# modelo.fit(x, etiquetas_discretizadas)

# predicciones = modelo.predict(x)
# score = modelo.score(x, etiquetas_discretizadas)

# print(predicciones)
# print(score)

In [727]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import confusion_matrix

# # Discretiza las etiquetas
# etiquetas_discretizadas = LabelEncoder().fit_transform(y)

# X_train, X_test, y_train, y_test = train_test_split(X, etiquetas_discretizadas, test_size=0.2, random_state=42)
# lg_rg = LogisticRegression()
# lg_rg.fit(X_train, y_train)
# predicc = lg_rg.predict(X_test)
# print(accuracy_score(y_test,predicc))
# confusion_matrix(y_test,predicc)

In [743]:
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.linear_model import LinearRegression

# poly = PolynomialFeatures(degree=3)
# x_polinomio = poly.fit_transform(X_train)

# model = LinearRegression()
# model.fit(x_polinomio, y_train)

# pred = model.predict(x_polinomio)
# score = model.score(x_polinomio, y_train)

# print(pred)
# print(score)

(912, 124)

### 3. Asignar el modelo (vacío) a una variable


In [733]:
from sklearn.linear_model import Lasso

alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
interaciones = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
for interacion in interaciones:
    model = Lasso(alpha=0.7, max_iter=interacion)
    model.fit(X_train,y_train)

    pred = model.predict(X_test)
    score = model.score(X_test, y_test)

    # print(pred)
    print(f"{interacion}: {score}")

100: 0.7563636932052724
200: 0.7563636932052724
300: 0.7563636932052724
400: 0.7563636932052724
500: 0.7563636932052724
600: 0.7563636932052724
700: 0.7563636932052724
800: 0.7563636932052724
900: 0.7563636932052724
1000: 0.7563636932052724


In [739]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
interaciones = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

for iteracion in interaciones:
    model = SVR(kernel='linear', max_iter=iteracion)
    model.fit(X_train, y_train)

    pred = model.predict(X_test)
    score = np.sqrt(mean_squared_error(y_test,pred))

    # print(pred)
    print(score)


c:\Users\cgdo2\Documents\Master Data\DS_PT_09_2023\.venv\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\cgdo2\Documents\Master Data\DS_PT_09_2023\.venv\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\cgdo2\Documents\Master Data\DS_PT_09_2023\.venv\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=300).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\cgdo2\Documents\Master Data\DS_PT_09_2023\.venv\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=400).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


3220.8029770691724
906.4265449074891
1019.158181651842
1210.736245813494
871.9138854962424
1764.5836833188512
544.1833548636961
510.84333183236635
865.4019820040626
520.8332304032142


c:\Users\cgdo2\Documents\Master Data\DS_PT_09_2023\.venv\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=900).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\cgdo2\Documents\Master Data\DS_PT_09_2023\.venv\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [758]:
from sklearn.tree import DecisionTreeRegressor

criterion: ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']

interaciones = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30]

for iteracion in interaciones:

    model = DecisionTreeRegressor(criterion='squared_error', max_depth=iteracion, random_state=5)

    model.fit(X_train, y_train)

    pred = model.predict(X_test)
    score = np.sqrt(mean_squared_error(y_test,pred))

    # print(pred)
    print(score)

# Mirar itertools

583.1418247245762
519.5380869267545
473.86639743457744
507.57220516978776
440.98454339956595
449.0792978254957
476.96376107754065
487.59118239826836
488.6290100412841
452.3048923019882
461.2499114884387
449.2398343707833
459.23155282297046
459.23155282297046


In [780]:
model.feature_importances_

df = pd.DataFrame(model.feature_importances_)

df2 = pd.concat([df, pd.DataFrame(train2.columns)], axis=1)
df2.columns=["features", "name"]
df2.sort_values('features' , ascending=False)

,features,name
2,5.503212e-01,Ram
3,9.812304e-02,Weight
36,5.190463e-02,TypeName_Netbook
0,4.512469e-02,laptop_ID
12,4.106093e-02,cpu_core
10,3.822959e-02,Touchscreen
5,3.356534e-02,main_storage_type
9,2.706966e-02,screen_ips
38,1.693597e-02,TypeName_Ultrabook
1,1.639800e-02,Inches


In [696]:
# from sklearn.linear_model import Ridge

# model = Ridge(alpha=0.0001)
# model.fit(X_train,y_train)

# pred = model.predict(X_test)
# score = model.score(X_test, y_test)

# # print(pred)
# print(score)

In [697]:
# from sklearn.linear_model import SGDRegressor

# model = SGDRegressor(penalty="l2", alpha=0.1, tol=None,  max_iter=1000, eta0=0.01, random_state=42)
# model.fit(X_train,y_train)

# pred = model.predict(X_test)
# score = model.score(X_test, y_test)

# # print(pred)
# print(score)

In [698]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

Porcentaje de error del modelo

In [699]:
scores = cross_val_score(model, X, y, cv=5)

np.mean(scores)

0.7562704349668142

El modelo se va a equivocar en estos precios

In [700]:
scores = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')

np.mean(scores)

-333.16799450539463

In [701]:
model.fit(X_train,y_train)

LinearRegression()

In [702]:
pred = model.predict(X_test)
pred

array([1332.00630106, -442.64238658,  804.93871381, 1662.66176569,
       2144.12747082,  436.27986517, 1043.68149389, 2068.42692605,
        883.39100039, 1246.65351855, 1683.27043577,  584.72639803,
        382.4230617 , 1497.26211252, 1394.99891533, 1736.74448445,
       1773.07146639, 1211.64603747,  806.51321173,  433.7581506 ,
        608.87285393,  492.14886421, 1056.34508266, 1106.45816518,
        861.57062885,  416.24278366, 1022.98961016, 1907.68239733,
        840.78286041,  840.39825077, 1508.53903792, 1045.14501289,
        784.98452047, 1639.7539698 , 1017.65863939,  235.62052264,
        570.78998731,  998.3852292 , 1351.2792558 , 1234.91975115,
       1240.16262249, 2065.80788015, 1320.07309952, 3159.39885986,
       1160.38183266,  318.5506173 , 1183.47345404,  534.34741088,
       1078.44215264,  685.76692838, 1568.7492598 ,  834.31312413,
        216.65566686, 2887.64013862, 1237.18791196, 1641.72391152,
       1702.24227219, 1066.47279641, 1964.61716545, 1778.37859

### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de MAE.

In [703]:
from sklearn.metrics import mean_squared_error

print('RMSE :', np.sqrt(mean_squared_error(y_test,pred)))

RMSE : 353.91358947488015


In [606]:
pd.options.display.max_rows = None
pd.DataFrame(model.coef_, X_train.columns, columns=['Coefficient'])


,Coefficient
laptop_ID,1.404152e-01
Inches,-3.995916e+01
Ram,2.348279e+02
Weight,9.443195e+01
screen_ips,7.739209e+00
Touchscreen,-2.335566e+01
Resolution,1.111666e+02
main_storage_type,8.637840e+01
main_storage_capacity,2.944894e+01
secondary_storage_type,-2.887912e+01


-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 2. Carga los datos de `test.csv` para predecir.

In [250]:
X_pred = pd.read_csv("data/test.csv")
X_pred.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,750,Toshiba,Tecra X40-D-10G,Notebook,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.25kg
1,726,Mediacom,Smartbook 142,Notebook,14.0,IPS Panel Full HD 1920x1200,Intel Atom Z8350 1.92GHz,4GB,32GB HDD,Intel HD Graphics 400,Windows 10,1.4kg
2,633,Mediacom,SmartBook 140,Notebook,14.0,Full HD 1920x1080,Intel Atom x5-Z8350 1.44GHz,2GB,32GB Flash Storage,Intel HD Graphics,Windows 10,1.4kg
3,363,Dell,Inspiron 7577,Gaming,15.6,Full HD 1920x1080,Intel Core i5 7300HQ 2.5GHz,8GB,1TB HDD,Nvidia GeForce GTX 1050,Windows 10,2.65kg
4,319,Asus,VivoBook Flip,2 in 1 Convertible,11.6,Touchscreen 1366x768,Intel Celeron Dual Core N3350 1.1GHz,2GB,32GB Flash Storage,Intel HD Graphics 500,Windows 10,1.1kg


**¿Por qué da error?** 

In [251]:
X_pred = limpiar_dataframe(X_pred)

C:\Users\cgdo2\AppData\Local\Temp\ipykernel_75352\2141475305.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["dedicated_gpu"] = df['Gpu'].str.contains(r"(Nvidia|AMD)")


In [252]:
train3, X_pred = apply_onehot_encoder(train, ["Company", "TypeName", "OpSys", "cpu_brand"], X_pred)

c:\Users\cgdo2\Documents\Master Data\DS_PT_09_2023\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [159]:
# x_polinomio_nuevo = poly.fit_transform(X_pred)

# predictions_submit = model.predict(x_polinomio_nuevo)
# predictions_submit

In [253]:
predictions_submit = model.predict(X_pred)
predictions_submit

array([-1.79983427e+21, -1.99982666e+21, -1.79984442e+21, -1.79982063e+21,
       -9.10587478e+20, -1.79985332e+21, -1.79982690e+21, -1.79987206e+21,
       -1.79983622e+21, -1.79986401e+21, -1.79985360e+21, -1.79984794e+21,
       -1.79987592e+21, -9.10583094e+20, -1.79985627e+21, -1.79983401e+21,
       -1.79985065e+21, -1.79983672e+21, -1.79981672e+21, -1.79980515e+21,
       -1.79982757e+21, -1.79988353e+21, -9.10547848e+20, -9.10556923e+20,
       -9.10570416e+20, -3.19971924e+21, -9.10530813e+20, -1.79979088e+21,
       -1.79988366e+21, -1.79987002e+21, -1.79988788e+21, -1.79987258e+21,
       -1.79982823e+21, -1.79985712e+21, -1.79983661e+21, -1.79984110e+21,
       -1.79987056e+21, -1.79983075e+21, -9.10518205e+20, -1.79982727e+21,
       -1.79986540e+21, -1.79988936e+21, -9.10587906e+20, -3.55527436e+21,
       -1.79985264e+21, -1.79986968e+21, -1.79982765e+21, -1.79987224e+21,
       -1.79982668e+21, -1.79981289e+21, -1.79981761e+21, -1.79982263e+21,
       -1.79982307e+21, -

IMPORTANTE: APLICAR LO MISMO A ESTOS DATOS QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [160]:
# ¿Qué opináis?
# Sí

![](https://www.tierraljelechu.com/web/img/wiki_up/1.996-SorpresaDto.-1-Red.jpg)

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [254]:
sample = pd.read_csv("data/sample_submission.csv")

In [255]:
sample.head()

,laptop_ID,Price_euros
0,750,500
1,726,500
2,633,500
3,363,650
4,319,650


In [256]:
sample.shape

(391, 2)

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [257]:
submission = pd.DataFrame({"laptop_ID": sample['laptop_ID'], "Price_euros": predictions_submit})

In [258]:
submission.head()

,laptop_ID,Price_euros
0,750,-1.799834e+21
1,726,-1.999827e+21
2,633,-1.799844e+21
3,363,-1.799821e+21
4,319,-9.105875e+20


In [223]:
submission.shape

(391, 2)

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [167]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [231]:
chequeator(submission)

You're ready to submit!
